In [1]:
import pandas as pd  
adr = pd.read_csv("address_clust.csv")
adr.head()

,address_id,cluster_id
0,71413451,1
1,71411914,1
2,71410369,1
3,71410100,1
4,71410040,1


In [2]:
adr.count()[0]

6996

In [3]:
adr1 = adr[adr['cluster_id'] == 1]
adr1['cluster_id'].describe()

count    3652.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: cluster_id, dtype: float64

In [4]:
adr2 = adr[adr['cluster_id'] == 2]
adr2['cluster_id'].describe()

count    3344.0
mean        2.0
std         0.0
min         2.0
25%         2.0
50%         2.0
75%         2.0
max         2.0
Name: cluster_id, dtype: float64

In [5]:
tr = pd.read_csv("address_stats.csv") 
tr.head()

,id,address_id,transaction_id,received,sent
0,87134765,46402336,19162323,0.0,1800000.0
1,87134766,45919235,19162323,0.0,1071052.0
2,87134767,46529090,19162323,1000000.0,0.0
3,87134768,46529091,19162323,1821052.0,0.0
4,87154706,46529090,19166856,0.0,1000000.0


In [6]:
tr["sent"].describe().apply(lambda x: format(x, 'f'))

count          76724.000000
mean         3521806.596319
std         80598839.672983
min                0.000000
25%                0.000000
50%             5500.000000
75%          1345500.000000
max      20071687710.000000
Name: sent, dtype: object

In [7]:
tr["received"].describe().apply(lambda x: format(x, 'f'))

count          76724.000000
mean         3518724.881263
std         80861454.260771
min                0.000000
25%                0.000000
50%                0.000000
75%          1346000.000000
max      20010181916.000000
Name: received, dtype: object

In [8]:
# Выберем записи для 1, 2 и 0 кластеров
adr1_rec = tr[(tr.received > 0) & (tr.address_id.isin(adr1.address_id))] # получено первым кластером
adr2_rec = tr[(tr.received > 0) & (tr.address_id.isin(adr2.address_id))] # получено вторым кластером
adr0_rec = tr[(tr.received > 0) & (~tr.address_id.isin(adr1.address_id)) & (~tr.address_id.isin(adr2.address_id))] # получено нулевым кластером
adr1_sen = tr[(tr.sent > 0) & (tr.address_id.isin(adr1.address_id))] # отправлено первым кластером
adr2_sen = tr[(tr.sent > 0) & (tr.address_id.isin(adr2.address_id))] # отправлено вторым кластером
adr0_sen = tr[(tr.sent > 0) & (~tr.address_id.isin(adr1.address_id)) & (~tr.address_id.isin(adr2.address_id))] # получено нулевым кластером


In [9]:
print (tr[(tr.received > 0)].count()[0], adr1_rec.count()[0], adr2_rec.count()[0], adr0_rec.count()[0], 
    "\n",tr[(tr.sent > 0)].count()[0], adr1_sen.count()[0], adr2_sen.count()[0], adr0_sen.count()[0])

37722 16806 12106 8810 
 39002 16143 13420 9439


In [10]:
print ("Received clusters processed", not tr[(tr.received > 0)].count()[0] - (adr1_rec.count()[0] + adr2_rec.count()[0] + adr0_rec.count()[0]), 
       "\nSent clusters processed", not tr[(tr.sent > 0)].count()[0] - (adr1_sen.count()[0] + adr2_sen.count()[0] + adr0_sen.count()[0]))

Received clusters processed True 
Sent clusters processed True


In [11]:
# Первый - второй
# trsum_1to2 = adr1_sen[adr1_sen['transaction_id'].isin(adr2_rec['transaction_id'])]
trsum_1to2_rec = adr2_rec[adr2_rec['transaction_id'].isin(adr1_sen['transaction_id'])]
print ("Сумма BTC, отправленных 1-ым кластером во 2ой: <= {}".format((
    trsum_1to2_rec['received'].sum() / (10**8))))
#trsum_2to1 = adr2_sen[adr2_sen['transaction_id'].isin(adr1_rec['transaction_id'])]
trsum_2to1_rec = adr1_rec[adr1_rec['transaction_id'].isin(adr2_sen['transaction_id'])]
print ("Сумма BTC, отправленных 2-ым кластером в 1ый: <= {}".format((
    trsum_2to1_rec['received'].sum() / (10**8))))

# Первый - нулевой
# trsum_1to0 = adr1_sen[adr1_sen['transaction_id'].isin(adr0_rec['transaction_id'])]
trsum_1to0_rec = adr0_rec[adr0_rec['transaction_id'].isin(adr1_sen['transaction_id'])]
print ("\nСумма BTC, отправленных 1-ым кластером во 0ой: <= {}".format((
    trsum_1to0_rec['received'].sum() / (10**8))))
# trsum_0to1 = adr0_sen[adr0_sen['transaction_id'].isin(adr1_rec['transaction_id'])]
trsum_0to1_rec = adr1_rec[adr1_rec['transaction_id'].isin(adr0_sen['transaction_id'])]
print ("Сумма BTC, отправленных 0-ым кластером в 1ый: <= {}".format((
    trsum_0to1_rec['received'].sum() / (10**8))))

# Второй - нулевой
#trsum_2to0 = adr2_sen[adr2_sen['transaction_id'].isin(adr0_rec['transaction_id'])]
trsum_2to0_rec = adr0_rec[adr0_rec['transaction_id'].isin(adr2_sen['transaction_id'])]
print ("\nСумма BTC, отправленных 2-ым кластером во 0ой: <= {}".format((
    trsum_2to0_rec['received'].sum() / (10**8))))
#trsum_0to2 = adr0_sen[adr0_sen['transaction_id'].isin(adr2_rec['transaction_id'])]
trsum_0to2_rec = adr2_rec[adr2_rec['transaction_id'].isin(adr0_sen['transaction_id'])]
print ("Сумма BTC, отправленных 0-ым кластером в 2ой: <= {}".format((
    trsum_0to2_rec['received'].sum() / (10**8))))

# Fee 1ого кластера
fee1 = adr1_sen['sent'].sum() * ((tr[(tr.transaction_id.isin(adr1_sen.transaction_id))]['sent'].sum() 
        / tr[(tr.transaction_id.isin(adr1_sen.transaction_id))]['received'].sum()) - 1) / (10**8)
print (f"\nКомиссия, отправленная первым кластером равна {fee1} BTC", )

# Fee 2ого кластера
fee1 = adr2_sen['sent'].sum() * ((tr[(tr.transaction_id.isin(adr2_sen.transaction_id))]['sent'].sum() 
        / tr[(tr.transaction_id.isin(adr2_sen.transaction_id))]['received'].sum()) - 1) / (10**8)
print (f"\nКомиссия, отправленная вторым кластером равна {fee1} BTC", )


Сумма BTC, отправленных 1-ым кластером во 2ой: <= 129.24324
Сумма BTC, отправленных 2-ым кластером в 1ый: <= 135.82

Сумма BTC, отправленных 1-ым кластером во 0ой: <= 300.55392888
Сумма BTC, отправленных 0-ым кластером в 1ый: <= 297.11968388

Сумма BTC, отправленных 2-ым кластером во 0ой: <= 4.86984
Сумма BTC, отправленных 0-ым кластером в 2ой: <= 9.74013861

Комиссия, отправленная первым кластером равна 0.667839876692493 BTC

Комиссия, отправленная вторым кластером равна 0.7462067844594951 BTC


In [12]:
# в каких-то транзакциях есть одинаковые записи вплоть до id.
tr[tr['transaction_id'] == 21645495]

,id,address_id,transaction_id,received,sent
9619,98646384,52491535,21645495,0.0,1035000.0
9620,98646384,52491535,21645495,0.0,1035000.0
9621,98646385,50727409,21645495,0.0,3000000.0
9622,98646385,50727409,21645495,0.0,3000000.0
9623,98646386,52090595,21645495,0.0,3000000.0
9624,98646386,52090595,21645495,0.0,3000000.0
9625,98646387,49435192,21645495,0.0,3000000.0
9626,98646387,49435192,21645495,0.0,3000000.0
9627,98646388,49435647,21645495,0.0,3000000.0
9628,98646388,49435647,21645495,0.0,3000000.0


In [13]:
#Дропнем их
tr.drop_duplicates(inplace=True)
tr[tr['transaction_id'] == 21645495]

,id,address_id,transaction_id,received,sent
9619,98646384,52491535,21645495,0.0,1035000.0
9621,98646385,50727409,21645495,0.0,3000000.0
9623,98646386,52090595,21645495,0.0,3000000.0
9625,98646387,49435192,21645495,0.0,3000000.0
9627,98646388,49435647,21645495,0.0,3000000.0
9629,98646389,52488195,21645495,12011000.0,0.0
9631,98646390,52495555,21645495,1014000.0,0.0


In [14]:
## Повторим вычисления для нового датафрейма
# Выберем записи для 1, 2 и 0 кластеров
adr1_rec = tr[(tr.received > 0) & (tr.address_id.isin(adr1.address_id))] # получено первым кластером
adr2_rec = tr[(tr.received > 0) & (tr.address_id.isin(adr2.address_id))] # получено вторым кластером
adr0_rec = tr[(tr.received > 0) & (~tr.address_id.isin(adr1.address_id)) & (~tr.address_id.isin(adr2.address_id))] # получено нулевым кластером
adr1_sen = tr[(tr.sent > 0) & (tr.address_id.isin(adr1.address_id))] # отправлено первым кластером
adr2_sen = tr[(tr.sent > 0) & (tr.address_id.isin(adr2.address_id))] # отправлено вторым кластером
adr0_sen = tr[(tr.sent > 0) & (~tr.address_id.isin(adr1.address_id)) & (~tr.address_id.isin(adr2.address_id))] # получено нулевым кластером


In [15]:
print (tr[(tr.received > 0)].count()[0], adr1_rec.count()[0], adr2_rec.count()[0], adr0_rec.count()[0], 
    "\n",tr[(tr.sent > 0)].count()[0], adr1_sen.count()[0], adr2_sen.count()[0], adr0_sen.count()[0])

26887 12099 6715 8073 
 27558 12096 6715 8747


In [16]:
print ("Received clusters processed", not tr[(tr.received > 0)].count()[0] - (adr1_rec.count()[0] + adr2_rec.count()[0] + adr0_rec.count()[0]), 
       "\nSent clusters processed", not tr[(tr.sent > 0)].count()[0] - (adr1_sen.count()[0] + adr2_sen.count()[0] + adr0_sen.count()[0]))

Received clusters processed True 
Sent clusters processed True


In [17]:
# Первый - второй
# trsum_1to2 = adr1_sen[adr1_sen['transaction_id'].isin(adr2_rec['transaction_id'])]
trsum_1to2_rec = adr2_rec[adr2_rec['transaction_id'].isin(adr1_sen['transaction_id'])]
print ("Сумма BTC, отправленных 1-ым кластером во 2ой: <= {}".format((
    trsum_1to2_rec['received'].sum() / (10**8))))
#trsum_2to1 = adr2_sen[adr2_sen['transaction_id'].isin(adr1_rec['transaction_id'])]
trsum_2to1_rec = adr1_rec[adr1_rec['transaction_id'].isin(adr2_sen['transaction_id'])]
print ("Сумма BTC, отправленных 2-ым кластером в 1ый: <= {}".format((
    trsum_2to1_rec['received'].sum() / (10**8))))

# Первый - нулевой
# trsum_1to0 = adr1_sen[adr1_sen['transaction_id'].isin(adr0_rec['transaction_id'])]
trsum_1to0_rec = adr0_rec[adr0_rec['transaction_id'].isin(adr1_sen['transaction_id'])]
print ("\nСумма BTC, отправленных 1-ым кластером во 0ой: <= {}".format((
    trsum_1to0_rec['received'].sum() / (10**8))))
# trsum_0to1 = adr0_sen[adr0_sen['transaction_id'].isin(adr1_rec['transaction_id'])]
trsum_0to1_rec = adr1_rec[adr1_rec['transaction_id'].isin(adr0_sen['transaction_id'])]
print ("Сумма BTC, отправленных 0-ым кластером в 1ый: <= {}".format((
    trsum_0to1_rec['received'].sum() / (10**8))))

# Второй - нулевой
#trsum_2to0 = adr2_sen[adr2_sen['transaction_id'].isin(adr0_rec['transaction_id'])]
trsum_2to0_rec = adr0_rec[adr0_rec['transaction_id'].isin(adr2_sen['transaction_id'])]
print ("\nСумма BTC, отправленных 2-ым кластером во 0ой: <= {}".format((
    trsum_2to0_rec['received'].sum() / (10**8))))
#trsum_0to2 = adr0_sen[adr0_sen['transaction_id'].isin(adr2_rec['transaction_id'])]
trsum_0to2_rec = adr2_rec[adr2_rec['transaction_id'].isin(adr0_sen['transaction_id'])]
print ("Сумма BTC, отправленных 0-ым кластером в 2ой: <= {}".format((
    trsum_0to2_rec['received'].sum() / (10**8))))

# Fee 1ого кластера
fee1 = adr1_sen['sent'].sum() * ((tr[(tr.transaction_id.isin(adr1_sen.transaction_id))]['sent'].sum() 
        / tr[(tr.transaction_id.isin(adr1_sen.transaction_id))]['received'].sum()) - 1) / (10**8)
print (f"\nКомиссия, отправленная первым кластером равна {fee1} BTC", )

# Fee 2ого кластера
fee1 = adr2_sen['sent'].sum() * ((tr[(tr.transaction_id.isin(adr2_sen.transaction_id))]['sent'].sum() 
        / tr[(tr.transaction_id.isin(adr2_sen.transaction_id))]['received'].sum()) - 1) / (10**8)
print (f"\nКомиссия, отправленная вторым кластером равна {fee1} BTC", )


Сумма BTC, отправленных 1-ым кластером во 2ой: <= 64.62162
Сумма BTC, отправленных 2-ым кластером в 1ый: <= 67.91

Сумма BTC, отправленных 1-ым кластером во 0ой: <= 291.5108346
Сумма BTC, отправленных 0-ым кластером в 1ый: <= 288.7905886

Сумма BTC, отправленных 2-ым кластером во 0ой: <= 2.43507
Сумма BTC, отправленных 0-ым кластером в 2ой: <= 6.09640861

Комиссия, отправленная первым кластером равна 0.5027980741478584 BTC

Комиссия, отправленная вторым кластером равна 0.373228573081378 BTC
